## 函數定義-爬取網頁

In [ ]:
def getdetail(url , title, category, cur):
    res = requests.get(url)
    soup = BeautifulSoup(res.text)
    dic = {"summary":soup.select('#summary')[0].text,
           "title": title,
           "category": category
    }
    sql = "insert into news_main({}) values({})"
    sql2 = sql.format(','.join(dic.keys()), ','.join(len(dic) * '?'))
    cur.execute(sql2, dic.values())

## 資料存進 sqlite

In [ ]:
import requests
import sqlite3 
con = sqlite3.connect('news.sqlite')
cur = con.cursor()
cur.execute("""drop table news_main;""")
cur.execute("""create table news_main(summary text, title text, category varchar(20));""")
con.commit()
from bs4 import BeautifulSoup
domain = 'http://www.appledaily.com.tw'
for page in range(1,30):
    res = requests.get('http://www.appledaily.com.tw/realtimenews/section/new/' + str(page))
    soup = BeautifulSoup(res.text)
    for rtddt in soup.select('.rtddt'):
        title = rtddt.select('h1')[0].text
        category = rtddt.select('h2')[0].text
        if domain not in rtddt.select('a')[0]['href'] :
            url = domain + rtddt.select('a')[0]['href'] 
        else:
            url = rtddt.select('a')[0]['href'] 
        getdetail(url , title, category, cur)
        con.commit()
con.close()

## jieba 斷詞demostrate

In [ ]:
import jieba
jieba.add_word('周子瑜')
for ele in jieba.cut('韓國女團「TWICE」成員、16歲台灣正妹周子瑜，回台參加學力鑑定考試並與親友師長會面後，結束短短不到2天行程，今清晨5時35分在隨行人員陪同下抵達高雄小港機場，5時39分進入出境管制門，將搭乘上午7時華航CI0164班機直飛韓國仁川機場，預計上午10時45分抵達'):
    print ele,

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

ary = ['【更新】柯P：洪智坤洩漏公文案還沒看到公文　今處理',
'留洪智坤 柯：殘障求職不易',
'人事處議處洪智坤　柯P：不清楚議處結果']
corpus = []
for title in ary:
    corpus.append('|'.join(jieba.cut(title)))
print corpus[0]  
print corpus[1]  
print corpus[2]  
vectorizer = CountVectorizer() 
X = vectorizer.fit_transform(corpus)
word = vectorizer.get_feature_names() 
for w in word:
    print w,
print 
print X.toarray()

## 讀取資料庫，抓幾個類別進行測試

In [ ]:
import sqlite3 
import jieba
con = sqlite3.connect('news.sqlite')
cur = con.cursor()
data = cur.execute("""select * from news_main;""")
corpus = []
tags = []
for rec in data:
    if (rec[2] == '娛樂'.decode('utf-8')) or \
        (rec[2] == '社會'.decode('utf-8')) or \
        (rec[2] == '財經'.decode('utf-8')):
        summary = rec[0]
        corpus.append(' '.join(jieba.cut(summary)))
        tags.append(rec[2]) 

con.close()

In [ ]:
#使用 jieba 斷詞並結合 CountVectorizer 進行字詞的矩陣計算(有出現的自詞+1)
import jieba
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer() 
X = vectorizer.fit_transform(corpus)
word = vectorizer.get_feature_names() 

In [ ]:
#進行資料訓練 (70%訓練 30%測試 )
from sklearn.cross_validation import train_test_split
train_data, test_data, train_tag, test_tag = train_test_split(X, tags, test_size=0.30, random_state=42)

In [ ]:
#引進 naive_bayes 演算法，進行運算，看是否能正確分類『每篇新聞』屬於『哪一個類別』
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB(alpha=0.01)   # naive_bayes 是用機率在計算，在這邊設定，即使沒有出現的字詞也給他 alpha=0.01 因為機率若為 0 會有問題
clf.fit(train_data,train_tag)

In [ ]:
#進行預測
pred = clf.predict(test_data)

In [ ]:
#計算演算法正確率
from sklearn.metrics import accuracy_score
print accuracy_score(test_tag, pred)

In [ ]:
#印出演算法分類結果
from sklearn.metrics import confusion_matrix 
a = confusion_matrix(test_tag, pred) 
for ele in clf.classes_:
    print ele,
print
print a

In [ ]:
#印出字詞出現次數，與分類相關資料
index = 0
coef_features_c1_c2_c3 = []
import re
for feat, c1, c2, c3 in zip(vectorizer.get_feature_names(), \
                        clf.feature_count_[0], clf.feature_count_[1], clf.feature_count_[2]):
    coef_features_c1_c2_c3.append(tuple([clf.coef_[0][index], feat, c1, c2, c3]))
    index+=1
        
for i in sorted(coef_features_c1_c2_c3):
    if i[2] == max(i[2], i[3],i[4]) and i[2] >0 and  re.match(u"[\u4e00-\u9fa5]+",i[1]):
        print "娛樂", i[1], i[2], i[3], i[4]